In [1]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('../3D-ResNets-PyTorch/')
sys.path.append('./important_csvs/')

from helpers_3d import *
from helpers_training import *
from focal_loss_2 import *
from load_data_and_augmentations import *
from imbalanced_sampler_3 import MultilabelBalancedRandomSampler


In [2]:
tensor_transform = get_tensor_transform('Kinetics', False)
train_spat_transform = get_spatial_transform(1)
train_temp_transform = va.TemporalFit(size=16)
#train_temp_transform = get_temporal_transform(16)
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/centre_Ch2/'
df = pd.read_csv('./important_csvs/more_balanced_dataset/big_stratified_new.csv')

####################################################
bs = 20
df_train = get_df(df, 50, True, False, False)
class_image_paths, end_idx, idx_label = get_indices(df_train, root_dir)
seq_length = 50
indices, labels = get_final_indices_train(idx_label, end_idx, 'overlap', skip_frames=15, set_step=25, seq_length=seq_length, per_label=True)
indices = torch.cat(indices)
indices = indices[torch.randperm(len(indices))]
labels = []
for i in class_image_paths:
    labels.append(i[2])
labels = np.array(labels)
train_sampler = MultilabelBalancedRandomSampler(
    labels, indices, class_choice="least_sampled"
)
dataset = MyDataset(
        image_paths = class_image_paths,
        seq_length = seq_length,
        temp_transform = train_temp_transform,
        spat_transform = train_spat_transform,
        tensor_transform = tensor_transform,
        length = len(train_sampler),
        lstm = False,
        oned = False,
        augment = True,
        multi = 1)
train_loader = DataLoader(
        dataset,
        batch_size = bs,
        sampler = train_sampler,
        drop_last = True,
        num_workers = 0)
##########################################################################################

#train_loader = get_loader(16, 64, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)
df_valid = get_df(df, 50, False, True, False)
class_image_paths, end_idx, idx_label= get_indices(df_valid, root_dir)
indices, labels = get_final_indices_valid(idx_label, end_idx, 'overlap', skip_frames=15, set_step=25, seq_length=seq_length, per_label=True)
valid_loader, valid_dataset = get_loader_new(seq_length, bs, indices, class_image_paths, valid_temp_transform,
                                             valid_spat_transform, tensor_transform, False, False, True, 1)


In [7]:
img, lab = next(iter(valid_loader))

In [8]:
img.shape

torch.Size([20, 3, 16, 576, 704])

In [10]:
len(train_loader)*20

3280

In [11]:
len(valid_loader)*20

900

In [6]:
223/45

4.955555555555556

In [4]:
len(train_loader)*20

3280